In [ ]:
# pip install -r requirements.txt

In [1]:
from docling.document_converter import DocumentConverter
# import cv2

source = "https://arxiv.org/pdf/2408.09869"  # file path or URL
converter = DocumentConverter()
doc = converter.convert(source).document
print(doc.export_to_markdown())  # output: "### Docling Technical Report[...]"

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-15 07:36:53,977 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 07:36:53,996 - INFO - Going to convert document batch...
2025-12-15 07:36:53,997 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-15 07:36:54,008 - INFO - Loading plugin 'docling_defaults'
2025-12-15 07:36:54,010 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-15 07:36:54,026 - INFO - Loading plugin 'docling_defaults'
2025-12-15 07:36:54,029 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-15 07:36:54,030 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-12-15 07:36:54,030 - INFO

<!-- image -->

## Docling Technical Report

## Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research R¨ uschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge vari

In [3]:
%%writefile system_resources.py
import psutil
import torch

def check_system_resources():
    cpu_count = psutil.cpu_count(logical=False)
    cpu_percent = psutil.cpu_percent(interval=0.2)

    gpu_available = torch.cuda.is_available()
    gpu_mem_free = None

    if gpu_available:
        free, total = torch.cuda.mem_get_info()
        gpu_mem_free = free // (1024 ** 2)  # MB

    return {
        "cpu_count": cpu_count,
        "cpu_percent": cpu_percent,
        "gpu_available": gpu_available,
        "gpu_mem_free_mb": gpu_mem_free
    }


Writing system_resources.py


In [4]:
%%writefile worker.py
from docling.document_converter import DocumentConverter

def convert_to_markdown(source: str, use_gpu: bool) -> str | None:
    try:
        converter = DocumentConverter()
        doc = converter.convert(source).document
        return doc.export_to_markdown()
    except Exception as e:
        return f"[ERROR] {source}: {e}"


Writing worker.py


In [ ]:
%%writefile parallel_exec.py
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
from system_resources import check_system_resources
from worker import convert_to_markdown

BASE_OUTPUT_DIR = "output/docling"

def create_storage():
    uid = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(BASE_OUTPUT_DIR, uid)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir, uid

def init_storage(run_dir: str, sources: list[str]):
    output_path = os.path.join(run_dir, "results.json")
    data = {
        "meta": {
            "total": len(sources),
            "start_time": datetime.now().isoformat()
        },
        "documents": []
    }
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return output_path

def append_result(output_json: str, record: dict):
    with open(output_json, "r+", encoding="utf-8") as f:
        data = json.load(f)
        data["documents"].append(record)
        f.seek(0)
        json.dump(data, f, ensure_ascii=False, indent=2)
        f.truncate()

def convert_list(sources: list[str], batch_size: int = 4):

    run_dir, uid = create_storage()
    output_json = init_storage(run_dir, sources)

    print(f"Output folder: {run_dir}")

    resources = check_system_resources()
    cpu_count = resources["cpu_count"]
    gpu_available = resources["gpu_available"]

    max_workers = min(cpu_count, batch_size)

    print(f"Starting conversion with {max_workers} workers")
    print(f"GPU available: {gpu_available}")

    results = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(convert_to_markdown, src, gpu_available): src
            for src in sources
        }

        for idx, future in enumerate(as_completed(futures), 1):
            src = futures[future]
            try:
                content = future.result()
                record = {
                    "source": src,
                    "status": "success",
                    "content": content
                }
                results.append(result)
                print(f"[{idx}/{len(sources)}] Done: {src}")
            except Exception as e:
                record = {
                    "source": src,
                    "status": "error",
                    "error": str(e)
                }
                print(f"Error processing {src}: {e}")
            
            append_result(output_json, record)

            # resource check mỗi 3 file
            if idx % 3 == 0:
                res = check_system_resources()
                if res["cpu_percent"] > 90:
                    print("⚠ CPU HIGH USAGE detected")

                if res["gpu_available"] and res["gpu_mem_free_mb"] < 2048:
                    print("⚠ GPU memory low → future jobs still on CPU")

    return results


Overwriting main.py


In [6]:
# import os
import psutil
import concurrent.futures
import torch
import time
from docling.document_converter import DocumentConverter

def check_system_resources():
    """Kiểm tra tài nguyên hệ thống, đặc biệt là CPU và GPU."""
    # Kiểm tra CPU
    cpu_count = psutil.cpu_count(logical=False)  # số lõi CPU thực
    cpu_percent = psutil.cpu_percent(interval=1)  # Tỉ lệ sử dụng CPU trong 1 giây
    
    # Kiểm tra GPU (nếu có)
    gpu_available = torch.cuda.is_available()
    gpu_count = torch.cuda.device_count() if gpu_available else 0
    gpu_name = torch.cuda.get_device_name(0) if gpu_available else None
    gpu_memory = torch.cuda.memory_allocated(0) if gpu_available else 0
    
    print(f"CPU: {cpu_count} cores, {cpu_percent}% used")
    if gpu_available:
        print(f"GPU: {gpu_name}, {gpu_count} devices, {gpu_memory // (1024 * 1024)} MB memory used")
    else:
        print("No GPU available")
    
    return cpu_count, cpu_percent, gpu_available, gpu_memory

def convert_to_markdown(source:str,gpu_available: bool):
    """Chuyển đổi từng tài liệu thành markdown"""
    try:
        converter = DocumentConverter()
        if gpu_available:
            print(f"Processing {source} on GPU...")
        else:
            print(f"Processing {source} on CPU...")
        doc = converter.convert(source).document
        return doc.export_to_markdown()
    except Exception as e:
        print(f"Error converting {source}: {str(e)}")
        return None

def convertList(sources:list[str]):
    """Hàm chính để xử lý danh sách các tài liệu song song"""
    cpu_count, cpu_percent, gpu_available, gpu_memory = check_system_resources() # Kiểm tra tài nguyên hệ thống
    max_workers = cpu_count # Sử dụng tối đa số worker bằng số lõi CPU có sẵn (may be change into gpu_count, however i use cpu at that time)
    
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i, result in enumerate(executor.map(lambda source: convert_to_markdown(source, gpu_available), sources)):
            results.append(result)
            
            
            print(f"Completed processing document {i + 1}") # Kiểm tra lại tài nguyên sau mỗi lần xử lý
            cpu_count, cpu_percent, gpu_available, gpu_memory = check_system_resources()

            
            if cpu_percent > 90: # Điều chỉnh số worker nếu CPU hoặc GPU memory bị quá tải
                print("CPU usage is high, reducing number of workers.")
                max_workers = max(1, cpu_count // 2)  # Giảm worker
                executor._max_workers = max_workers  # Cập nhật worker
            elif gpu_available and gpu_memory < 2 * 1024 * 1024 * 1024:  # Kiểm tra nếu bộ nhớ GPU dưới 2GB
                print("GPU memory is low, switching to CPU.")
                gpu_available = False  # Chuyển qua sử dụng CPU

            time.sleep(1) # Để tránh kiểm tra quá nhanh, thêm một khoảng dừng nhỏ

    return results

In [ ]:
# Ví dụ sử dụng
sources = [
    "https://arxiv.org/pdf/2408.09869",  # Paper 1
    "https://arxiv.org/pdf/2311.04155",  # Paper 2
    "https://arxiv.org/pdf/1706.03762",  # Paper 3
    "https://arxiv.org/pdf/2302.09664",  # Paper 4
    "https://arxiv.org/pdf/2003.12771",  # Paper 5
    "https://arxiv.org/pdf/1910.02707",  # Paper 6
    "https://arxiv.org/pdf/1705.04510",  # Paper 7
    "https://arxiv.org/pdf/1506.01497",  # Paper 8
    "https://arxiv.org/pdf/1802.05365",  # Paper 9
    "https://arxiv.org/pdf/2004.09602",  # Paper 10
    "https://arxiv.org/pdf/1602.02235",  # Paper 11
    "https://arxiv.org/pdf/1801.06538",  # Paper 12
    "https://arxiv.org/pdf/1611.09060",  # Paper 13
    "https://arxiv.org/pdf/1912.05483",  # Paper 14
    "https://arxiv.org/pdf/1709.08624",  # Paper 15
    "https://arxiv.org/pdf/1810.04805",  # Paper 16
    "https://arxiv.org/pdf/1502.03167",  # Paper 17
    "https://arxiv.org/pdf/1807.02547",  # Paper 18
    "https://arxiv.org/pdf/2103.00634",   # Paper 19
    "https://arxiv.org/pdf/2103.00634"
]


# CPU times: user 294 ms, sys: 198 ms, total: 492 ms
# Wall time: 13min 30s

In [ ]:
%%time
from parallel_exec import convert_list

converted_docs = convert_list(sources)

# In kết quả markdown của từng tài liệu
for i, doc in enumerate(converted_docs):
    if doc:
        print(f"Document {i + 1}:")
        print(doc)

🚀 Starting conversion with 2 workers
GPU available: False


2025-12-15 07:47:41,242 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 07:47:41,260 - INFO - Going to convert document batch...
2025-12-15 07:47:41,262 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-15 07:47:41,269 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 07:47:41,278 - INFO - Loading plugin 'docling_defaults'
2025-12-15 07:47:41,283 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-15 07:47:41,288 - INFO - Going to convert document batch...
2025-12-15 07:47:41,290 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-15 07:47:41,300 - INFO - Loading plugin 'docling_defaults'
2025-12-15 07:47:41,306 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-15 07:47:41,307 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-12-15 07

✅ [1/20] Done: https://arxiv.org/pdf/2408.09869


[INFO] 2025-12-15 07:49:03,951 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 07:49:03,952 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 07:49:03,956 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 07:49:03,957 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 07:49:04,067 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 07:49:04,069 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 07:49:04,094 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-12-15 07:49:04,097 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OC

✅ [2/20] Done: https://arxiv.org/pdf/1706.03762


...

✅ [18/20] Done: https://arxiv.org/pdf/2103.00634


[INFO] 2025-12-15 08:00:47,434 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:00:47,436 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:00:47,440 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:00:47,441 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:00:47,560 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:00:47,562 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:00:47,586 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-12-15 08:00:47,593 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OC

✅ [19/20] Done: https://arxiv.org/pdf/1807.02547


2025-12-15 08:01:11,328 - INFO - Finished converting document 2103.00634v4.pdf in 24.43 sec.


✅ [20/20] Done: https://arxiv.org/pdf/2103.00634
Document 1:
<!-- image -->

## Docling Technical Report

## Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research R¨ uschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format 

In [ ]:
%%time
from worker import convert_to_markdown

for src in sources:
    doc = convert_to_markdown(src,False)
    print(doc)

2025-12-15 08:01:54,959 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 08:01:54,983 - INFO - Going to convert document batch...
2025-12-15 08:01:54,984 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-15 08:01:55,005 - INFO - Loading plugin 'docling_defaults'
2025-12-15 08:01:55,007 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-15 08:01:55,030 - INFO - Loading plugin 'docling_defaults'
2025-12-15 08:01:55,035 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-15 08:01:55,036 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-12-15 08:01:55,037 - INFO - easyocr cannot be used because it is not installed.
2025-12-15 08:01:55,206 - INFO - Accelerator device: 'cpu'
[INFO] 2025-12-15 08:01:55,226 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:01:55,229 [RapidOCR] device_config.py:50: Usi

2025-12-15 08:02:39,473 - INFO - Finished converting document 2408.09869v5.pdf in 44.63 sec.
2025-12-15 08:02:39,601 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-15 08:02:39,604 - INFO - Going to convert document batch...
2025-12-15 08:02:39,605 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-15 08:02:39,606 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-12-15 08:02:39,606 - INFO - easyocr cannot be used because it is not installed.
2025-12-15 08:02:39,607 - INFO - Accelerator device: 'cpu'
[INFO] 2025-12-15 08:02:39,625 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:02:39,626 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:02:39,638 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 08:02:39,639 [RapidOCR] main.p

<!-- image -->

## Docling Technical Report

## Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research R¨ uschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge vari

[INFO] 2025-12-15 08:02:39,824 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:02:39,825 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:02:39,827 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:02:39,827 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:02:39,915 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:02:39,916 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:02:39,939 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-12-15 08:02:39,940 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OC

## Black-Box Prompt Optimization: Aligning Large Language Models without Model Training

Jiale Cheng 1 , 2 * , Xiao Liu 3 , 2 * , Kehan Zheng 1 , Pei Ke 1 , Hongning Wang 1 , Yuxiao Dong 3 , Jie Tang 3 , Minlie Huang 1 †

1 The Conversational Artificial Intelligence (CoAI) Group, Tsinghua University 2 Zhipu AI

3

The Knowledge Engineering Group (KEG), Tsinghua University chengjl23@mails.tsinghua.edu.cn, shawliu9@gmail.com, aihuang@tsinghua.edu.cn

## Abstract

Large language models (LLMs) have shown impressive success in various applications. However, these models are often not well aligned with human intents, which calls for additional treatments on them; that is, the alignment problem. To make LLMs better follow user instructions, existing alignment methods primarily focus on further training them. However, the extra training of LLMs is usually expensive in terms of GPU computing; even worse, some LLMs are not accessible for userdemanded training, such as GPTs. In this work, we take

...

[INFO] 2025-12-15 08:07:58,837 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:07:58,838 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:07:58,840 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:07:58,840 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:07:58,942 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:07:58,943 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:07:58,965 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-12-15 08:07:58,966 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OC

1

## INTRODUCTION

Recent advances in object detection are driven by the success of region proposal methods ( e.g ., [4]) and region-based convolutional neural networks (RCNNs) [5]. Although region-based CNNs were computationally expensive as originally developed in [5], their cost has been drastically reduced thanks to sharing convolutions across proposals [1], [2]. The latest incarnation, Fast R-CNN [2], achieves near real-time rates using very deep networks [3], when ignoring the time spent on region proposals . Now, proposals are the test-time computational bottleneck in state-of-the-art detection systems.

Region proposal methods typically rely on inexpensive features and economical inference schemes. Selective Search [4], one of the most popular methods, greedily merges superpixels based on engineered low-level features. Yet when compared to efficient detection networks [2], Selective Search is an order of magnitude slower, at 2 seconds per image in a CPU implementation. EdgeBox

[INFO] 2025-12-15 08:09:08,952 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:09:08,953 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:09:08,955 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:09:08,955 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:09:09,035 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:09:09,036 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:09:09,059 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-12-15 08:09:09,059 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OC

## Deep contextualized word representations

Matthew E. Peters † , Mark Neumann † , Mohit Iyyer † , Matt Gardner † ,

{ matthewp,markn,mohiti,mattg } @allenai.org

Christopher Clark ∗ , Kenton Lee ∗ , Luke Zettlemoyer †∗

{ csquared,kentonl,lsz } @cs.washington.edu

†

Paul G. Allen School of Computer Science &amp; Engineering, University of Washington

Allen Institute for Artificial Intelligence ∗

## Abstract

We introduce a new type of deep contextualized word representation that models both (1) complex characteristics of word use (e.g., syntax and semantics), and (2) how these uses vary across linguistic contexts (i.e., to model polysemy). Our word vectors are learned functions of the internal states of a deep bidirectional language model (biLM), which is pretrained on a large text corpus. We show that these representations can be easily added to existing models and significantly improve the state of the art across six challenging NLP problems, including question answering, textual

[INFO] 2025-12-15 08:09:56,500 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:09:56,502 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:09:56,504 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:09:56,505 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 08:09:56,590 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 08:09:56,592 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2025-12-15 08:09:56,615 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-12-15 08:09:56,616 [RapidOCR] main.py:50: Using /usr/local/python/3.12.1/lib/python3.12/site-packages/rapidocr/models/ch_PP-OC